In [76]:
import sys
import os
import re
import pandas as pd
from torchnlp.datasets import imdb_dataset
from nltk.corpus import stopwords
import torch as t
from torch import nn

In [77]:
data_path = r'E:\kaggle\movies'
train_data_path = os.path.join(data_path,'labeledTrainData.tsv')
test_data_path = os.path.join(data_path,'testData.tsv')


In [78]:
# Load the imdb training dataset
train = imdb_dataset(train=True)
train[0]  # RETURNS: {'text': 'For a movie that gets..', 'sentiment': 'pos'}

{'sentiment': 'pos',
 'text': 'Bromwell High is a cartoon comedy. It ran at the same time as some other programs about school life, such as "Teachers". My 35 years in the teaching profession lead me to believe that Bromwell High\'s satire is much closer to reality than is "Teachers". The scramble to survive financially, the insightful students who can see right through their pathetic teachers\' pomp, the pettiness of the whole situation, all remind me of the schools I knew and their students. When I saw the episode in which a student repeatedly tried to burn down the school, I immediately recalled ......... at .......... High. A classic line: INSPECTOR: I\'m here to sack one of your teachers. STUDENT: Welcome to Bromwell High. I expect that many adults of my age think that Bromwell High is far fetched. What a pity that it isn\'t!'}

In [79]:
import pandas as pd
df_imdb = pd.DataFrame(list(train))
df_imdb.count()

sentiment    25000
text         25000
dtype: int64

In [80]:
df_imdb['sentiment'] = df_imdb['sentiment'].map({'pos':1,'neg':0})

In [81]:
set(df_imdb['sentiment'])

{0, 1}

In [82]:
df_imdb.head()

,sentiment,text
0,1,Bromwell High is a cartoon comedy. It ran at t...
1,1,Homelessness (or Houselessness as George Carli...
2,1,Brilliant over-acting by Lesley Ann Warren. Be...
3,1,This is easily the most underrated film inn th...
4,1,This is not the typical Mel Brooks film. It wa...


In [83]:
s = df_imdb['text'][0]

In [84]:
def prossed(i,tol):
    if(i%(tol//100)==0):
        print(i//(tol//100),end=' ')
i = 0
stopw = set(stopwords.words('english'))
def clean_text(text):
    text = re.sub(r'[^\w\'\s]','',text)
    text = text.lower()
    text = text.split(' ')
    text = [word for word in text if not word in stopw]
    text = ' '.join(text)
    global i
    i = i+1
    prossed(i,25000)
    return text
df_imdb['prossed_reviews'] = df_imdb.text.apply(clean_text)

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 

In [85]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

max_features = 6000
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(df_imdb['prossed_reviews'])
list_tokenized_train = tokenizer.texts_to_sequences(df_imdb['prossed_reviews'])


In [86]:
print(list_tokenized_train[0])
print('len',len(list_tokenized_train[0]))
df_imdb.prossed_reviews.apply(lambda x: len(x.split(" "))).mean()
maxlen = 125
X_t = pad_sequences(list_tokenized_train, maxlen=maxlen)
print(X_t.shape)
y = df_imdb['sentiment']
len(y)

[211, 941, 119, 1934, 9, 5661, 288, 41, 5191, 4856, 59, 4760, 5867, 357, 150, 1832, 13, 2202, 521, 5191, 1888, 5790, 1351, 12, 104, 1085, 113, 767, 2782, 5662, 542, 1351, 106, 269, 1335, 3445, 642, 3395, 288, 1055, 211, 238, 251, 2808, 54, 4, 5191, 1335, 2275, 211, 392, 30, 1324, 456, 26, 211, 131, 2136]
len 58
(25000, 125)


25000

In [88]:
max_features = 6000
embed_size=300
hidden_size=256
class mvNet(nn.Module):
    def __init__(self):
        super(mvNet,self).__init__()
        self.embd = nn.Embedding(max_features,embed_size)
        self.lstm = nn.LSTM(embed_size,hidden_size,batch_first=True)
        self.linear1 = nn.Linear(hidden_size,128)
        self.linear2 = nn.Linear(128,1)
    
    def forward(self,x):
        x = self.embd(x)
        x,_ = self.lstm(x)
        x = x[:,-1,:].squeeze()
        m = nn.Sigmoid()
        x = m(self.linear1(x))
        x = m(self.linear2(x))
        return x

In [89]:
net = mvNet()
from torch import optim
criterion = nn.BCELoss() # 交叉熵损失函数
optimizer = optim.Adam(net.parameters(), lr=0.001)
t.save (net.state_dict(), os.path.join(data_path,'lstm.pk1')) # 保存的是以字典 key - value pair 形式的数据，每一个参数对应着一个值 state_dict 状态字典 

In [90]:
from torch.utils import data
from torch.utils.data import DataLoader
class Movie(data.Dataset):
    def __init__(self, x,y):
        self.x = x
        self.y = y
    def __getitem__(self, index):
        data = self.x[index]
        label = self.y[index]
        return data, label
    
    def __len__(self):
        return len(self.y)

movdata = Movie(X_t,y)
trainloader = DataLoader(movdata,batch_size=50, shuffle=True, num_workers=0, drop_last=False)

t.set_num_threads(8)
for epoch in range(2):  
    
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        
        # 输入数据
        inputs, labels = data
        inputs = inputs.long()
        labels = labels.float()
        # 梯度清零
        optimizer.zero_grad()
        outputs = net(inputs)
        outputs = outputs.squeeze()
        loss = criterion(outputs, labels)
        loss.backward()   
        
        # 更新参数 
        optimizer.step()
        
        # 打印log信息
        # loss 是一个scalar,需要使用loss.item()来获取数值，不能使用loss[0]
        running_loss += loss.item()
        if i % 10 == 9: # 每2000个batch打印一下训练状态
            print('[%d, %5d] loss: %.3f' \
                  % (epoch+1, i+1, running_loss / 10))
            running_loss = 0.0
print('Finished Training')
# 模型参数保存
t.save (net.state_dict(), os.path.join(data_path,'lstm.pk1')) # 保存的是以字典 key - value pair 形式的数据，每一个参数对应着一个值 state_dict 状态字典 


[1,    10] loss: 0.695
[1,    20] loss: 0.689
[1,    30] loss: 0.674
[1,    40] loss: 0.624
[1,    50] loss: 0.674
[1,    60] loss: 0.653
[1,    70] loss: 0.639
[1,    80] loss: 0.633
[1,    90] loss: 0.563
[1,   100] loss: 0.521
[1,   110] loss: 0.514
[1,   120] loss: 0.553
[1,   130] loss: 0.518
[1,   140] loss: 0.538
[1,   150] loss: 0.519
[1,   160] loss: 0.476
[1,   170] loss: 0.507
[1,   180] loss: 0.539
[1,   190] loss: 0.515
[1,   200] loss: 0.499
[1,   210] loss: 0.554
[1,   220] loss: 0.532
[1,   230] loss: 0.492
[1,   240] loss: 0.500
[1,   250] loss: 0.500
[1,   260] loss: 0.469
[1,   270] loss: 0.447
[1,   280] loss: 0.501
[1,   290] loss: 0.444
[1,   300] loss: 0.456
[1,   310] loss: 0.490
[1,   320] loss: 0.491
[1,   330] loss: 0.460
[1,   340] loss: 0.433
[1,   350] loss: 0.462
[1,   360] loss: 0.578
[1,   370] loss: 0.534
[1,   380] loss: 0.516
[1,   390] loss: 0.489
[1,   400] loss: 0.507
[1,   410] loss: 0.448
[1,   420] loss: 0.446
[1,   430] loss: 0.471
[1,   440] 

In [91]:
net.load_state_dict(t.load(os.path.join(data_path,'lstm.pk1')))
optimizer = optim.SGD(net.parameters(), lr=0.001)
t.set_num_threads(8)
for epoch in range(2):  
    
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        
        # 输入数据
        inputs, labels = data
        inputs = inputs.long()
        labels = labels.float()
        # 梯度清零
        optimizer.zero_grad()
        # forward + backward 
        outputs = net(inputs)
        outputs = outputs.squeeze()
        loss = criterion(outputs, labels)
        loss.backward()   
        
        # 更新参数 
        optimizer.step()
        
        # 打印log信息
        # loss 是一个scalar,需要使用loss.item()来获取数值，不能使用loss[0]
        running_loss += loss.item()
        if i % 10 == 9: # 每2000个batch打印一下训练状态
            print('[%d, %5d] loss: %.3f' \
                  % (epoch+1, i+1, running_loss / 10))
            running_loss = 0.0
print('Finished Training')

[1,    10] loss: 0.255
[1,    20] loss: 0.251
[1,    30] loss: 0.287
[1,    40] loss: 0.311
[1,    50] loss: 0.300
[1,    60] loss: 0.316
[1,    70] loss: 0.297
[1,    80] loss: 0.328
[1,    90] loss: 0.303
[1,   100] loss: 0.284
[1,   110] loss: 0.279
[1,   120] loss: 0.286
[1,   130] loss: 0.323
[1,   140] loss: 0.306
[1,   150] loss: 0.296
[1,   160] loss: 0.280
[1,   170] loss: 0.260
[1,   180] loss: 0.288
[1,   190] loss: 0.268
[1,   200] loss: 0.276
[1,   210] loss: 0.266
[1,   220] loss: 0.320
[1,   230] loss: 0.278
[1,   240] loss: 0.263
[1,   250] loss: 0.308
[1,   260] loss: 0.279
[1,   270] loss: 0.262
[1,   280] loss: 0.281
[1,   290] loss: 0.285
[1,   300] loss: 0.276
[1,   310] loss: 0.240
[1,   320] loss: 0.264
[1,   330] loss: 0.293
[1,   340] loss: 0.251
[1,   350] loss: 0.281
[1,   360] loss: 0.260
[1,   370] loss: 0.269
[1,   380] loss: 0.272
[1,   390] loss: 0.284
[1,   400] loss: 0.292
[1,   410] loss: 0.266
[1,   420] loss: 0.233
[1,   430] loss: 0.271
[1,   440] 

In [92]:
import numpy as np
test_df = pd.read_csv(os.path.join(data_path,"testData.tsv"), header=0, sep='\t')
test_df['pro_review'] = test_df.review.apply(clean_text)
list_tokenized_train = tokenizer.texts_to_sequences(test_df['pro_review'])

2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 

In [93]:
maxlen = 125
X_t = pad_sequences(list_tokenized_train, maxlen=maxlen)
y = np.array([])
for i in range(250):
    tmp = net(t.tensor(X_t[i*100:i*100+100]).long())
    tmp = tmp.squeeze().data.numpy()
    tmp = np.array(tmp>0.5,dtype=np.int)
    y = np.concatenate((y,tmp))
    
lstm_df=pd.DataFrame({'id':test_df['id'],'sentiment':y})
lstm_df.to_csv(os.path.join(data_path,"lstm_result.csv"),index=False)